In [1]:
import sys
sys.path.append("../src/")
from loss_function import loss_function
from loss_function import cs
from torch.autograd import Variable
from evaluate import Evaluation
from cnn import CNN
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sklearn
import random
import time

In [2]:
import data_reader as dr

In [3]:
corpus_path = "../data/text_tokenized.txt.gz"
corpus = dr.read_corpus(corpus_path)

In [4]:
embedding_path = "../data/vectors_pruned.200.txt.gz"
embedding_tensor, word_to_indx = dr.getEmbeddingTensor(embedding_path)

In [5]:
ids_corpus = dr.map_corpus(corpus, word_to_indx)

In [6]:
train_path = "../data/train_random.txt"
train = dr.read_annotations(train_path)

In [7]:
train_ex = dr.create_train_set(ids_corpus, train)

In [52]:
import torch.utils.data

def train_model(train_data, dev_data, model):
    model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    model.train()

    lasttime = time.time()
    for epoch in range(1, 11):
        print("-------------\nEpoch {}:\n".format(epoch))

        loss = run_epoch(train_data, True, model, optimizer)
        #return loss
        print('Train loss: {:.6f}'.format(loss))
        torch.save(model, "model{}".format(epoch))
        
        (MAP, MRR, P1, P5) = run_epoch(dev_data, False, model, optimizer)
        print('Val MAP: {:.6f}, MRR: {:.6f}, P1: {:.6f}, P5: {:.6f}'.format(MAP, MRR, P1, P5))
        
        print('This epoch took: {:.6f}'.format(time.time() - lasttime))
        lasttime = time.time()

        
def run_epoch(data, is_training, model, optimizer):
    '''
    Train model for one pass of train data, and return loss, acccuracy
    '''
    data_loader = torch.utils.data.DataLoader(
        data,
        batch_size=40,
        shuffle=True,
        num_workers=4,
        drop_last=False)

    losses = []

    if is_training:
        model.train()
    else:
        model.eval()

    for batch in data_loader:
        pid_title = torch.unsqueeze(Variable(batch['pid_title']), 1)
        pid_body = torch.unsqueeze(Variable(batch['pid_body']), 1)
        rest_title = Variable(batch['rest_title'])
        rest_body = Variable(batch['rest_body'])
        
        pid_title_pad = torch.unsqueeze(Variable(batch['pid_title_pad']), 1)
        pid_body_pad = torch.unsqueeze(Variable(batch['pid_body_pad']), 1)
        rest_title_pad = Variable(batch['rest_title_pad'])
        rest_body_pad = Variable(batch['rest_body_pad'])
        
        pid_title, pid_body = pid_title.cuda(), pid_body.cuda()
        rest_title, rest_body = rest_title.cuda(), rest_body.cuda()
        pid_title_pad, pid_body_pad = pid_title_pad.cuda(), pid_body_pad.cuda()
        rest_title_pad, rest_body_pad = rest_title_pad.cuda(), rest_body_pad.cuda()
        
        if is_training:
            optimizer.zero_grad()
        
        pt = model(pid_title)
        pb = model(pid_body)
        rt = model(rest_title)
        rb = model(rest_body)
        
        # we need to take the mean pooling taking into account the padding
        # tensors are of dim batch_size x samples x output_size x (len - kernel + 1)
        # pad tensors are of dim batch_size x samples x (len - kernel + 1)
        
        pid_title_pad_ex = torch.unsqueeze(pid_title_pad, 2).expand_as(pt)
        pid_body_pad_ex = torch.unsqueeze(pid_body_pad, 2).expand_as(pb)
        rest_title_pad_ex = torch.unsqueeze(rest_title_pad, 2).expand_as(rt)
        rest_body_pad_ex = torch.unsqueeze(rest_body_pad, 2).expand_as(rb)
        
        pt = torch.squeeze(torch.sum(pt * pid_title_pad_ex, dim = 3), dim = 3)
        pb = torch.squeeze(torch.sum(pb * pid_body_pad_ex, dim = 3), dim = 3)
        rt = torch.squeeze(torch.sum(rt * rest_title_pad_ex, dim = 3), dim = 3)
        rb = torch.squeeze(torch.sum(rb * rest_body_pad_ex, dim = 3), dim = 3)

        # tensors are not of dim batch_size x samples x output_size
        # need to scale down because not all uniformly padded
        
        ptp_norm = torch.sum(pid_title_pad, dim = 2).clamp(min = 1).expand_as(pt)
        pbp_norm = torch.sum(pid_body_pad, dim = 2).clamp(min = 1).expand_as(pb)
        rtp_norm = torch.sum(rest_title_pad, dim = 2).clamp(min = 1).expand_as(rt)
        rbp_norm = torch.sum(rest_body_pad, dim = 2).clamp(min = 1).expand_as(rb)
        
        pt = pt / ptp_norm
        pb = pb / pbp_norm
        rt = rt / rtp_norm
        rb = rb / rbp_norm
        
        pid_tensor = (pt + pb)/2
        rest_tensor = (rt + rb)/2
        #pid_tensor = pb
        #rest_tensor = rb
        
        if is_training:
            loss = loss_function(pid_tensor, rest_tensor, margin = 0.2)
            loss.backward()
            losses.append(loss.cpu().data[0])
            optimizer.step()
        else:
            expanded = pid_tensor.expand_as(rest_tensor)
            similarity = cs(expanded, rest_tensor, dim=2).squeeze(2)
            similarity = similarity.data.cpu().numpy()
            labels = batch['labels'].numpy()
            l = dr.convert(similarity, labels)
            losses.extend(l)

    # Calculate epoch level scores
    if is_training:
        avg_loss = np.mean(losses)
        return avg_loss
    else:
        e = Evaluation(losses)
        MAP = e.MAP()*100
        MRR = e.MRR()*100
        P1 = e.Precision(1)*100
        P5 = e.Precision(5)*100
        return (MAP, MRR, P1, P5)

In [59]:
model = CNN(700, 3, embedding_tensor)

In [60]:
val_path = "../data/test.txt"
val = dr.read_annotations(val_path, K_neg = -1, prune_pos_cnt = -1)
val_ex = dr.create_dev_set(ids_corpus, val)

In [61]:
z = train_model(train_ex, val_ex, model)

-------------
Epoch 1:

Train loss: 0.134464
Val MAP: 54.621934, MRR: 68.544067, P1: 55.376344, P5: 41.827957
This epoch took: 86.605695
-------------
Epoch 2:

Train loss: 0.064663
Val MAP: 54.062740, MRR: 67.460312, P1: 53.763441, P5: 42.580645
This epoch took: 86.526572
-------------
Epoch 3:

Train loss: 0.041366
Val MAP: 54.520490, MRR: 67.550327, P1: 53.225806, P5: 42.473118
This epoch took: 86.589065
-------------
Epoch 4:

Train loss: 0.027097
Val MAP: 55.060882, MRR: 69.028646, P1: 55.376344, P5: 42.150538
This epoch took: 86.531316
-------------
Epoch 5:

Train loss: 0.018154
Val MAP: 55.287658, MRR: 69.060249, P1: 55.913978, P5: 43.118280
This epoch took: 86.441439
-------------
Epoch 6:

Train loss: 0.012213
Val MAP: 55.436991, MRR: 68.338148, P1: 53.763441, P5: 42.365591
This epoch took: 86.455577
-------------
Epoch 7:

Train loss: 0.008199
Val MAP: 55.487080, MRR: 68.252452, P1: 53.763441, P5: 43.440860
This epoch took: 86.426948
-------------
Epoch 8:

Train loss: 0.005

In [64]:
model = torch.load("model5")
real_val_path = "../data/dev.txt"
real_val = dr.read_annotations(real_val_path, K_neg = -1, prune_pos_cnt = -1)
real_val_ex = dr.create_dev_set(ids_corpus, real_val)
model = model.cuda()
z = run_epoch(real_val_ex, False, model, None)

In [65]:
z

(55.21228806480822, 67.10418592888246, 53.96825396825397, 45.714285714285744)